# Engine
> Execution spannerlog commands

In [ ]:
#| default_exp engine

In [ ]:
#| hide
from nbdev.showdoc import show_doc
from IPython.display import display, HTML
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from abc import ABC, abstractmethod
import pytest
from collections import defaultdict

import pandas as pd
from pathlib import Path
from typing import no_type_check, Set, Sequence, Any,Optional,List,Callable,Dict,Union
from pydantic import BaseModel
import networkx as nx
import itertools
import logging
logger = logging.getLogger(__name__)

from graph_rewrite import draw, draw_match, rewrite, rewrite_iter
from spannerlib.utils import (
    serialize_graph,
    serialize_df_values,
    checkLogs,
    get_new_node_name
    )

from spannerlib.span import Span,SpanParser
from spannerlib.data_types import (
    Var, 
    FreeVar, 
    RelationDefinition, 
    Relation, 
    IEFunction,
    IERelation, 
    Rule, 
    pretty
)
from spannerlib.ra import (
    _col_names,
    get_const,
    select,
    project,
    rename,
    union,
    intersection,
    difference,
    join,
    product,
    ie_map,
)

from spannerlib.term_graph import graph_compose, merge_term_graphs_pair,rule_to_graph,add_relation,add_project_uniq_free_vars



## Helper functions

In [ ]:
#| export
def _pd_drop_row(df,row_vals):
    new_df = df[(df!=row_vals).all(axis=1)]
    return new_df

def _pd_append_row(df,row_vals):
    return pd.concat([df,pd.DataFrame([row_vals],columns=df.columns)])

In [ ]:
df = pd.DataFrame([
    [1,'2fs'],[3,4]
])
assert list(_pd_drop_row(df,[3,4]).itertuples(index=False,name=None))==[(1,'2fs')]
assert list(_pd_append_row(df,[5,6]).itertuples(index=False,name=None)) == [(1, '2fs'), (3, 4), (5, 6)]

In [ ]:
#| export
class DB(dict):
    def __repr__(self):
        key_str=', '.join(self.keys())
        return f'DB({key_str})'

## Engine Class

In [ ]:
#| export
from copy import deepcopy
class Engine():
    def __init__(self,rewrites=None):
        if rewrites is None:
            self.rewrites = []
        self.symbol_table={
            # key : type,val
        }
        self.Relation_defs={
            # key : RelationDefinition for both real and derived relations
        }
        self.ie_functions={
            # name : IEFunction class
        }

        self.term_graph = nx.DiGraph()
        
        self.node_counter = itertools.count()
        self.rule_counter = itertools.count()

        self.db = DB(
            # relation_name: dataframe
        )

        # lets skip this for now and keep it a an attribute in the node graph
        self.rules_to_ids = {
            # rule pretty string, to node id in term_graph
        }

        # self.rels_to_nodes() = {
        #     # relation name to node that represents it
        # }
    

    def set_var(self,var_name,value,read_from_file=False):
        symbol_table = self.symbol_table
        if var_name in symbol_table:
            existing_type,existing_value = symbol_table[var_name]
            if type(value) != existing_type:
                raise ValueError(f"Variable {var_name} was previously defined with {existing_value}({pretty(existing_type)})"
                                f" but is trying to be redefined to {value}({pretty(type(value))}) of a different type which might interfere with previous rule definitions")    
        symbol_table[var_name] = type(value),value
        return
    def get_var(self,var_name):
        return self.symbol_table.get(var_name,None)
    
    def del_var(self,var_name):
        del self.symbol_table[var_name]

    def get_relation(self,rel_name:str):
        return self.Relation_defs.get(rel_name,None)

    def set_relation(self,rel_def:RelationDefinition):
        if rel_def.name in self.Relation_defs:
            existing_def = self.Relation_defs[rel_def.name]
            if existing_def != rel_def:
                raise ValueError(f"Relation {rel_def.name} was previously defined with {existing_def}"
                                f"but is trying to be redefined to {rel_def} which might interfere with previous rule definitions")
        else:
            self.Relation_defs[rel_def.name] = rel_def
            #TODO fix make sure that the empty df has the correct types based on the rel_def
            empty_df = pd.DataFrame(columns=_col_names(len(rel_def.scheme)))
            self.db[rel_def.name] = empty_df
            self.term_graph.add_node(rel_def.name,rel=rel_def.name)

    def del_relation(self,rel_name:str):
        # TODO we need to think about what to do with all relations that used this rule
        raise NotImplementedError("deleting relations is not supported yet")
        return

    def add_fact(self,fact:Relation):
        self.db[fact.name] = _pd_append_row(self.db[fact.name],fact.terms)

    def add_facts(self,rel_name,facts:pd.DataFrame):
        facts= facts.copy()
        facts.columns = _col_names(len(facts.columns))
        self.db[rel_name] = pd.concat([self.db[rel_name],facts])

    def del_fact(self,fact:Relation):
        self.db[fact.name] = _pd_drop_row(df = self.db[fact.name],row_vals=fact.terms)

    def get_ie_function(self,name:str):
        return self.ie_functions.get(name,None)

    def set_ie_function(self,ie_func:IEFunction):
        self.ie_functions[ie_func.name]=ie_func

    def del_ie_function(self,name:str):
        del self.ie_functions[name]

    def add_rule(self,rule:Rule,schema:RelationDefinition=None):
        if not self.get_relation(rule.head.name) and schema is None:
            raise ValueError(f"Relation {rule.head.name} not defined before adding the rule with it's head\n"
                             f"And an relation schema was not supplied."
                             f"existing relations are {self.Relation_defs.keys()}")

        if not schema is None:
            self.set_relation(schema)

        rule_id = next(self.rule_counter)

        self.rules_to_ids[pretty(rule)] = rule_id

        g2 = rule_to_graph(rule,rule_id)

        merge_term_graph = merge_term_graphs_pair(self.term_graph,g2)
        self.term_graph = merge_term_graph
        

    def del_rule(self,rule_str:str):
        if not rule_str in self.rules_to_ids:
            raise ValueError(f"Rule {rule_str} does not exist\n"
                             f"existing rules are {self.rules_to_ids.keys()}")
        rule_id = self.rules_to_ids[rule_str]
        self.rules_to_ids.pop(rule_str)
        g = self.term_graph
        nodes_to_delete=[]
        for u in g.nodes:
            node_rule_ids = g.nodes[u].get('rule_id',set())
            if rule_id in node_rule_ids:
                node_rule_ids.remove(rule_id)
                if len(node_rule_ids) == 0:
                    nodes_to_delete.append(u)
        g.remove_nodes_from(nodes_to_delete)
            
        return

    def _inline_db_and_ies_in_graph(self,g:nx.DiGraph):
        g=deepcopy(g)
        for u in g.nodes:
            if g.out_degree(u)==0 and 'rel' in g.nodes[u]:
                g.nodes[u]['op'] = 'get_rel'
                g.nodes[u]['db'] = self.db
            elif g.nodes[u]['op'] == 'ie_map':
                ie_func_name = g.nodes[u]['func']
                ie_definition = self.ie_functions[ie_func_name]
                g.nodes[u]['func'] = ie_definition.func
                g.nodes[u]['name'] = ie_definition.name
                g.nodes[u]['in_schema'] = ie_definition.in_schema
                g.nodes[u]['out_schema'] = ie_definition.out_schema
        return g


    def plan_query(self,q_rel:Relation,rewrites=None):
        if rewrites is None:
            rewrites = self.rewrites
        query_graph = self._inline_db_and_ies_in_graph(self.term_graph)

        # get the sub term graph induced by the relation head
        root_node = q_rel.name
        connected_nodes = list(nx.shortest_path(query_graph,root_node).keys())
        query_graph = nx.DiGraph(nx.subgraph(query_graph,connected_nodes))
        
        # add selects renames etc based on the query relation
        root_node,_ = add_relation(query_graph,name='query',terms=q_rel.terms,source=root_node)

        # TODO for all rewrites, run them
        return query_graph,root_node

    def execute_plan(self,query_graph,root_node,return_intermediate=False):
        results_dict = defaultdict(list)
        results = compute_node(query_graph,root_node,results_dict)
        if return_intermediate:
            return results,results_dict
        else:
            return results

    def run_query(self,q:Relation,rewrites=None,return_intermediate=False):
        query_graph,root_node = self.plan_query(q,rewrites)
        return self.execute_plan(query_graph,root_node,return_intermediate=return_intermediate)


### Test

In [ ]:
s = pd.DataFrame([
    [1,1],
    [2,2],
    [3,3],
    [4,5]
])

s2 = pd.DataFrame([
    [1,2,3],
    [2,3,4],
    [3,4,5],
    [4,5,6]
])

In [ ]:
r1 = Rule(
    head=Relation(name='R', terms=[FreeVar(name='X'), FreeVar(name='Y')]),
    body=[
        Relation(name='S', terms=[FreeVar(name='X'),FreeVar(name='Y')]),
        Relation(name='S2', terms=[FreeVar(name='X'), FreeVar(name='A'),3]),
    ])

r2 = Rule(
    head=Relation(name='R', terms=[FreeVar(name='X'), FreeVar(name='Y')]),
    body=[
        Relation(name='S', terms=[FreeVar(name='X'),FreeVar(name='Y')]),
        IERelation(name='T', in_terms=[FreeVar(name='X'),FreeVar(name='Y')], out_terms=[FreeVar(name='X'),FreeVar(name='Y')]),
    ])

r3 = Rule(
    head=Relation(name='R2', terms=[FreeVar(name='X'), FreeVar(name='Y')]),
    body=[
        Relation(name='S3', terms=[FreeVar(name='X'),FreeVar(name='Y')]),
        Relation(name='S2', terms=[FreeVar(name='X'), FreeVar(name='A'),1]),
    ])


rec_r1 = Rule(
    head=Relation(name='A', terms=[FreeVar(name='X'), FreeVar(name='Y')]),
    body=[
        Relation(name='B', terms=[FreeVar(name='X'),FreeVar(name='Y')]),
    ])

rec_r2 = Rule(
    head=Relation(name='A', terms=[FreeVar(name='X'), FreeVar(name='Y')]),
    body=[
        Relation(name='C', terms=[FreeVar(name='X'),FreeVar(name='Y')]),
    ])

rec_r3 = Rule(
    head=Relation(name='B', terms=[FreeVar(name='X'), FreeVar(name='Y')]),
    body=[
        Relation(name='D', terms=[FreeVar(name='X'),FreeVar(name='Y')]),
    ])

rec_r4 = Rule(
    head=Relation(name='B', terms=[FreeVar(name='X'), FreeVar(name='Y')]),
    body=[
        Relation(name='A', terms=[FreeVar(name='X'),FreeVar(name='Y')]),
    ])


In [ ]:
e = Engine()
e.set_relation(RelationDefinition(name='S', scheme=[int,int]))
e.set_relation(RelationDefinition(name='S2', scheme=[int,int,int]))
e.set_relation(RelationDefinition(name='S3', scheme=[int,int]))

e.add_rule(r1,RelationDefinition(name='R', scheme=[int,int]))
e.add_rule(r2,RelationDefinition(name='R', scheme=[int,int]))
e.add_rule(r3,RelationDefinition(name='R2', scheme=[int,int]))

In [ ]:
e.add_facts('S',s)
e.add_facts('S2',s2)

In [ ]:
e.db['S']

,col_0,col_1
0,1,1
1,2,2
2,3,3
3,4,5


In [ ]:
e.db['S2']

,col_0,col_1,col_2
0,1,2,3
1,2,3,4
2,3,4,5
3,4,5,6


In [ ]:
draw(e.term_graph)

In [ ]:
e.rules_to_ids

{'R(X,Y) <- S(X,Y),S2(X,A,3)': 0,
 'R(X,Y) <- S(X,Y),T(X,Y) -> (X,Y)': 1,
 'R2(X,Y) <- S3(X,Y),S2(X,A,1)': 2}

In [ ]:
e.del_rule(pretty(r3))

In [ ]:
draw(e.term_graph)

In [ ]:
# assert serialize_graph(e.term_graph) ==([('S', {'rel': 'S', 'rule_id': {0, 1}}),
#   ('S2', {'rel': 'S2', 'rule_id': {0}}),
#   ('R', {'rel': 'R', 'op': 'union', 'rule_id': {0, 1}}),
#   (0, {'op': 'rename', 'names': [(0, 'X'), (1, 'Y')], 'rule_id': {0}}),
#   (1, {'op': 'rename', 'names': [(0, 'X'), (1, 'A')], 'rule_id': {0}}),
#   (2, {'op': 'select', 'theta': [(2, 3)], 'rule_id': {0}}),
#   (3, {'op': 'join', 'rule_id': {0}}),
#   (4, {'op': 'project', 'on': ['X', 'Y'], 'rel': '_R_0', 'rule_id': {0}}),
#   (8, {'op': 'rename', 'names': [(0, 'X'), (1, 'Y')], 'rule_id': {1}}),
#   (9, {'op': 'project', 'on': ['X', 'Y'], 'rule_id': {1}}),
#   (10, {'op': 'calc', 'func': 'T', 'rule_id': {1}}),
#   (11, {'op': 'rename', 'names': [(0, 'X'), (1, 'Y')], 'rule_id': {1}}),
#   (6, {'op': 'join', 'rule_id': {1}}),
#   (7, {'op': 'project', 'on': ['X', 'Y'], 'rel': '_R_1', 'rule_id': {1}})],
#  [('R', 4, {}),
#   ('R', 7, {}),
#   (0, 'S', {}),
#   (1, 'S2', {}),
#   (2, 1, {}),
#   (3, 0, {}),
#   (3, 2, {}),
#   (4, 3, {}),
#   (8, 'S', {}),
#   (9, 8, {}),
#   (10, 9, {}),
#   (11, 10, {}),
#   (6, 8, {}),
#   (6, 11, {}),
#   (7, 6, {})])

In [ ]:
e.add_rule(rec_r1,RelationDefinition(name='A', scheme=[int,int]))
e.add_rule(rec_r2,RelationDefinition(name='A', scheme=[int,int]))
e.add_rule(rec_r3,RelationDefinition(name='B', scheme=[int,int]))
e.add_rule(rec_r4,RelationDefinition(name='B', scheme=[int,int]))


In [ ]:
draw(e.term_graph)

## Naive execution

A recursive least fixed point logic algorithm mimicing the bottom up evalutation.

In [ ]:
#| export

def get_rel(rel,db,**kwargs):
    # helper function to get the relation from the db for external relations
    return db[rel]

op_to_func = {
    'union':union,
    'intersection':intersection,
    'difference':difference,
    'select':select,
    'project':project,
    'rename':rename,
    'join':join,
    'ie_map':ie_map,
    'get_rel':get_rel,
    'get_const':get_const,
    'product':product,
}

In [ ]:
#| export
def compute_node(G,u,results_dict):
    """
    """
    global op_to_func
    u_data = G.nodes[u]
    if u_data.get('final',False):
        return results_dict[u][-1]
    

    children_ids = list(G.successors(u))
    children_data = [G.nodes[child_id] for child_id in children_ids]

    if len(children_ids)==0:
        all_children_final=True
        children_results=[]
        
    else:
        # this block helps avoid infinite recursion when we need to initialize the 0th iteration of a node in a cycle as the empty relation
        if u_data.get('visited',False):
            u_data['visited'] = True
            results_dict[u].append(pd.DataFrame())
            return pd.DataFrame()
        u_data['visited'] =True
        
        children_results = [compute_node(G,child_id,results_dict) for child_id in children_ids]
        all_children_final = all(child_data.get('final',False) for child_data in children_data)




    op_code = u_data['op']
    op_func = op_to_func[op_code]
    logger.debug(f'computing node {u} with op {op_code} and children results {children_results} and data {u_data}')
    current_results = op_func(*children_results,**u_data)
    results_iter = len(results_dict[u])
    logger.debug(f'node {u} results after iteration {results_iter} are {current_results}')
    results_dict[u].append(current_results)

    # if all children are final then we can mark this node as final
    if all_children_final:
        logger.debug(f'setting node {u} as final since all children are final')
        u_data['final'] = True
    # else check for fixed point
    elif len(results_dict[u])>1:
        this_res = results_dict[u][-1]
        last_res = results_dict[u][-2]
        if pd.DataFrame.equals(this_res,last_res):
            logger.debug(f'setting node {u} as final since we reached a fixed point')
            u_data['final'] = True
        else:
            u_data['final'] = False
    # if this is the first iteration we need to wait for the second one to check for fixed point so final is still false
    else:
        u_data['final']=False

    return current_results





#### Case1

In [ ]:
e = Engine()
e.set_relation(RelationDefinition(name='S', scheme=[int,int]))
e.set_relation(RelationDefinition(name='S2', scheme=[int,int,int]))

e.add_rule(r1,RelationDefinition(name='R', scheme=[int,int]))
e.add_rule(r2,RelationDefinition(name='R', scheme=[int,int]))

e.add_facts('S',s)
e.add_facts('S2',s2)

def func(x,y):
    return [(y,x)]

ie_def = IEFunction(name='T',func=func,in_schema=[int,int],out_schema=[int,int])

e.set_ie_function(ie_def)
g = e._inline_db_and_ies_in_graph(e.term_graph)
print(e.rules_to_ids)
display(s)
display(s2)
draw(g)

{'R(X,Y) <- S(X,Y),S2(X,A,3)': 0, 'R(X,Y) <- S(X,Y),T(X,Y) -> (X,Y)': 1}


,0,1
0,1,1
1,2,2
2,3,3
3,4,5


,0,1,2
0,1,2,3
1,2,3,4
2,3,4,5
3,4,5,6


In [ ]:
draw(g)

In [ ]:
add_relation(g,terms=[FreeVar(name='X'),FreeVar(name='Y')],source='R')

(21, 'R')

In [ ]:
draw(g)

In [ ]:
with checkLogs():
    results_dict = defaultdict(list)
    query_res = compute_node(g.copy(),'R',results_dict)
assert serialize_df_values(query_res) == {(1, 1), (2, 2), (3, 3)}
query_res

__main__ - DEBUG - computing node S with op get_rel and children results [] and data {'rel': 'S', 'rule_id': {0, 1}, 'op': 'get_rel', 'db': DB(S, S2, R)}
__main__ - DEBUG - node S results after iteration 0 are   col_0 col_1
0     1     1
1     2     2
2     3     3
3     4     5
__main__ - DEBUG - setting node S as final since all children are final
__main__ - DEBUG - computing node 0 with op rename and children results [  col_0 col_1
0     1     1
1     2     2
2     3     3
3     4     5] and data {'op': 'rename', 'names': [(0, 'X'), (1, 'Y')], 'rule_id': {0}, 'visited': True}
__main__ - DEBUG - node 0 results after iteration 0 are    X  Y
0  1  1
1  2  2
2  3  3
3  4  5
__main__ - DEBUG - setting node 0 as final since all children are final
__main__ - DEBUG - computing node 1 with op rename and children results [   X  Y
0  1  1
1  2  2
2  3  3
3  4  5] and data {'op': 'rename', 'names': [(0, 'X'), (1, 'Y')], 'rule_id': {0}, 'visited': True}
__main__ - DEBUG - node 1 results after it

__main__ - DEBUG - computing node 7 with op join and children results [   X  Y
0  1  1
1  2  2
2  3  3
3  4  5,    X  A
0  1  2] and data {'op': 'join', 'rule_id': {0}, 'visited': True}
__main__ - DEBUG - node 7 results after iteration 0 are    X  Y  A
0  1  1  2
__main__ - DEBUG - setting node 7 as final since all children are final
__main__ - DEBUG - computing node 8 with op project and children results [   X  Y  A
0  1  1  2] and data {'op': 'project', 'on': ['X', 'Y'], 'rel': '_R_0', 'rule_id': {0}, 'visited': True}
__main__ - DEBUG - node 8 results after iteration 0 are    X  Y
0  1  1
__main__ - DEBUG - setting node 8 as final since all children are final
__main__ - DEBUG - computing node 11 with op rename and children results [  col_0 col_1
0     1     1
1     2     2
2     3     3
3     4     5] and data {'op': 'rename', 'names': [(0, 'X'), (1, 'Y')], 'rule_id': {1}, 'visited': True}
__main__ - DEBUG - node 11 results after iteration 0 are    X  Y
0  1  1
1  2  2
2  3  3
3  4  

,0,1
0,1,1
2,2,2
3,3,3


In [ ]:
q,r = e.plan_query(Relation(name='R',terms=[FreeVar(name='X'),FreeVar(name='Y')]))
draw(q)

adding relations for name='R' terms=[FreeVar(name='X'), FreeVar(name='Y')]


In [ ]:
res = e.run_query(Relation(name='R',terms=[FreeVar(name='X'),FreeVar(name='Y')]))
assert serialize_df_values(res) == {(1, 1), (2, 2), (3, 3)}

adding relations for name='R' terms=[FreeVar(name='X'), FreeVar(name='Y')]


In [ ]:
q,root = e.plan_query(Relation(name='R',terms=[FreeVar(name='X'),3]))
draw(q)

adding relations for name='R' terms=[FreeVar(name='X'), 3]


In [ ]:
res = e.run_query(Relation(name='R',terms=[FreeVar(name='S'),3]))
assert serialize_df_values(res)=={(3,)}
assert list(res.columns) == ['S']

adding relations for name='R' terms=[FreeVar(name='S'), 3]


#### case 2

In [ ]:
e2 = Engine()
e2.set_relation(RelationDefinition(name='C', scheme=[int,int]))
e2.set_relation(RelationDefinition(name='D', scheme=[int,int]))

for rule in [rec_r1,rec_r2,rec_r3,rec_r4]:
    e2.add_rule(rule,RelationDefinition(name=rule.head.name, scheme=[int,int]))

e2.add_fact(Relation(name='C',terms=[1,2]))
e2.add_fact(Relation(name='D',terms=[3,4]))

g2 = e2._inline_db_and_ies_in_graph(e2.term_graph)
e2.rules_to_ids

{'A(X,Y) <- B(X,Y)': 0,
 'A(X,Y) <- C(X,Y)': 1,
 'B(X,Y) <- D(X,Y)': 2,
 'B(X,Y) <- A(X,Y)': 3}

In [ ]:
# with checkLogs():
query_res = e2.run_query(Relation(name='A',terms=[FreeVar(name='X'),FreeVar(name='Y')]))
assert serialize_df_values(query_res) == {(3, 4), (1, 2)}
query_res = e2.run_query(Relation(name='B',terms=[FreeVar(name='X'),FreeVar(name='Y')]))
assert serialize_df_values(query_res) == {(3, 4), (1, 2)}

adding relations for name='A' terms=[FreeVar(name='X'), FreeVar(name='Y')]
adding relations for name='B' terms=[FreeVar(name='X'), FreeVar(name='Y')]


In [ ]:
draw(g2)

### Case 3

In [ ]:
e = Engine()
e.set_relation(RelationDefinition(name='string', scheme=[str]))
e.add_fact(Relation(name='string',terms=['a']))
e.add_fact(Relation(name='string',terms=['aa']))
def func(str):
    yield (len(str),)

e.set_ie_function(IEFunction(name='Length',func=func,in_schema=[str],out_schema=[int]))

r = Rule(
    head=Relation(name='string_length', terms=[FreeVar(name='Str'), FreeVar(name='Len')]),
    body=[
        Relation(name='string', terms=[FreeVar(name='Str')]),
        IERelation(name='Length', in_terms=[FreeVar(name='Str')], out_terms=[FreeVar(name='Len')]),
    ])

e.add_rule(r,RelationDefinition(name='string_length', scheme=[str,int]))



g = e._inline_db_and_ies_in_graph(e.term_graph)
print(e.rules_to_ids)
draw(g)


{'string_length(Str,Len) <- string(Str),Length(Str) -> (Len)': 0}


In [ ]:
# with checkLogs():
res = e.run_query(Relation(name='string_length', terms=[FreeVar(name='Str'), FreeVar(name='Len')]))
assert serialize_df_values(res) == {('a', 1), ('aa', 2)}
res

adding relations for name='string_length' terms=[FreeVar(name='Str'), FreeVar(name='Len')]


,Str,Len
0,a,1
1,aa,2


In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()
     